<a href="https://colab.research.google.com/github/cantaruttim/Deltalake/blob/main/Deltalake_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deltalake # install the python library for delta
!mkdir /lake # create a directory call lake
!git clone https://github.com/cantaruttim/Deltalake # take the data from github
!cp -r /content/Deltalake/data /lake # copy csv files into the lake folder (posteriormente deixo em content)

fatal: destination path 'Deltalake' already exists and is not an empty directory.


In [ ]:
# Installing Apache Spark into Google Colab
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip -q install findspark

In [ ]:
import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import deltalake

In [ ]:
df = pd.read_csv("/content/lake/data/acoes.csv")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1152 non-null   object 
 1   BOVA11.SA  1152 non-null   float64
dtypes: float64(1), object(1)
memory usage: 18.1+ KB


In [8]:
df.head()

,Date,BOVA11.SA
0,2019-01-02,87.900002
1,2019-01-03,88.000000
2,2019-01-04,88.500000
3,2019-01-07,88.480003
4,2019-01-08,88.830002


In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BOVA11.SA,1152.0,103.390451,10.56926,61.139999,97.097498,104.269997,111.077499,125.75


Writing a pandas DataFrame in Delta:

In [ ]:
write_deltalake('content/lake/delta_table', df,
                description='BOVA11.SA delta_table with Pandas',
                partition_by =['BOVA11.SA'])

Reading a Delta Table with Pandas:

In [10]:
dataframe = DeltaTable('content/lake/delta_table').to_pandas()
dataframe.head()

,Date,BOVA11.SA
0,2019-01-10,90.419998
1,2019-01-11,90.379997
2,2019-01-15,90.610001
3,2019-01-29,92.190002
4,2019-01-18,92.610001


In [12]:
DeltaTable('content/lake/delta_table').version()

0

In [13]:
DeltaTable('content/lake/delta_table').version() + 1

1

# METADATA

In [15]:
DeltaTable('content/lake/delta_table')

DeltaTable()

In [16]:
print(DeltaTable('content/lake/delta_table').metadata())

Metadata(id: 0aad4008-a997-46fe-9087-f7c7716ec761, name: None, description: BOVA11.SA delta_table with Pandas, partition_columns: ['BOVA11.SA'], created_time: 1694176740753, configuration: {})


In [17]:
DeltaTable('content/lake/delta_table').schema().to_pyarrow()

Date: string
BOVA11.SA: double

In [18]:
DeltaTable('content/lake/delta_table').history()

[{'timestamp': 1694176740758,
  'operation': 'CREATE TABLE',
  'operationParameters': {'metadata': '{"configuration":{},"created_time":1694176740753,"description":"BOVA11.SA delta_table with Pandas","format":{"options":{},"provider":"parquet"},"id":"0aad4008-a997-46fe-9087-f7c7716ec761","name":null,"partition_columns":["BOVA11.SA"],"schema":{"fields":[{"metadata":{},"name":"Date","nullable":true,"type":"string"},{"metadata":{},"name":"BOVA11.SA","nullable":true,"type":"double"}],"type":"struct"}}',
   'location': 'file:///content/content/lake/delta_table',
   'mode': 'ErrorIfExists',
   'protocol': '{"minReaderVersion":1,"minWriterVersion":1}'},
  'clientVersion': 'delta-rs.0.13.1'}]

In [19]:
DeltaTable('content/lake/delta_table').get_add_actions(flatten=True).to_pandas()

,path,size_bytes,modification_time,data_change,partition.BOVA11.SA,num_records,null_count.Date,min.Date,max.Date
0,BOVA11.SA=90.41999816894533/0-c05ffd80-e738-47...,1902,2023-09-08 12:38:55.648,True,90.419998,1,0,2019-01-10,2019-01-10
1,BOVA11.SA=90.37999725341795/0-c05ffd80-e738-47...,1902,2023-09-08 12:38:55.653,True,90.379997,1,0,2019-01-11,2019-01-11
2,BOVA11.SA=90.61000061035156/0-c05ffd80-e738-47...,1902,2023-09-08 12:38:55.662,True,90.610001,1,0,2019-01-15,2019-01-15
3,BOVA11.SA=92.19000244140624/0-c05ffd80-e738-47...,1902,2023-09-08 12:38:55.671,True,92.190002,1,0,2019-01-29,2019-01-29
4,BOVA11.SA=92.61000061035156/0-c05ffd80-e738-47...,1902,2023-09-08 12:38:55.677,True,92.610001,1,0,2019-01-18,2019-01-18
...,...,...,...,...,...,...,...,...,...
946,BOVA11.SA=115.4000015258789/0-c05ffd80-e738-47...,1902,2023-09-08 12:39:00.749,True,115.400002,1,0,2023-06-15,2023-06-15
947,BOVA11.SA=114.70999908447266/0-c05ffd80-e738-4...,1902,2023-09-08 12:39:00.749,True,114.709999,1,0,2023-07-17,2023-07-17
948,BOVA11.SA=107.7699966430664/0-c05ffd80-e738-47...,1902,2023-09-08 12:39:00.749,True,107.769997,1,0,2022-02-03,2022-02-03
949,BOVA11.SA=106/0-c05ffd80-e738-4765-a9ca-7e62bc...,1902,2023-09-08 12:39:00.749,True,106.000000,1,0,2022-12-23,2022-12-23


Changed Data Feed - CDF

In [22]:
currently_version = DeltaTable("content/lake/delta_table").version()

files_currently_version = DeltaTable("content/lake/delta_table", version = currently_version).file_uris()
files_currently_version

['/content/content/lake/delta_table/BOVA11.SA=90.41999816894533/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=90.37999725341795/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=90.61000061035156/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=92.19000244140624/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=92.61000061035156/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=94/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=91.4000015258789/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=90.98999786376952/0-c05ffd80-e738-4765-a9ca-7e62bcecd770-0.parquet',
 '/content/content/lake/delta_table/BOVA11.SA=91.2699966430664/0-c05ffd80-e738-4765-a9ca-7e62bce

# TEST COM APACHE SPARK

In [23]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [24]:
import findspark
findspark.init()

In [25]:
### import Spark libraries
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

### spark package maven coordinates - in case you are loading more than just delta
spark_packages_list = [
    'io.delta:delta-core_2.12:2.2.0',
]
spark_packages = ",".join(spark_packages_list)

### SparkSession
spark = (
    SparkSession.builder
    .config("spark.jars.packages", spark_packages)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [26]:
df_spark = spark.read.format("delta").load("content/lake/delta_table")

In [27]:
df_spark.show()

+----------+------------------+
|      Date|         BOVA11.SA|
+----------+------------------+
|2020-11-10|             101.0|
|2021-12-20|             101.0|
|2023-04-20|             101.0|
|2022-01-27| 108.1999969482422|
|2022-11-24| 108.1999969482422|
|2019-04-03| 91.16000366210938|
|2019-04-18| 91.16000366210938|
|2020-11-13|100.83000183105467|
|2021-11-08|100.83000183105467|
|2021-04-14| 115.5999984741211|
|2022-03-31| 115.5999984741211|
|2021-03-16|109.69000244140624|
|2019-09-11|             99.75|
|2022-05-05|             101.5|
|2021-03-01|105.94000244140624|
|2020-12-15|111.76000213623048|
|2021-01-11|118.62000274658205|
|2021-12-08| 104.1999969482422|
|2021-03-22|110.69000244140624|
|2020-05-13|  75.1500015258789|
+----------+------------------+
only showing top 20 rows

